## Dram Item Cleaning

This notebook attempts to map Dram Shop items on to the actual beverage along with the producer and additional information. For instance, we want to map `Ommegang - Rare Vos Belgian Style Amber Ale  - 6.5% ABV 20 IBU` to `ommegang` (the producer), `Rare Vos Belgian Style Amber Ale` (the beer), and `6.5% ABV 20 IBU` (the additional information). 

This is hard! The items are manually entered so we have things like abbreviations, misspellings, odd orderings, etc. 


We'll start by pulling in the items and their categories, which will be our starting point.

In [1]:
import os
import re
import datetime 
from collections import Counter
import requests
import time

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

### GBQ Set Up

In this next section we connect to our GBQ project and list the data sets inside to test the connection.

In [2]:
# These first two values will be different on your machine. 
service_path = "/Users/chandler/Dropbox/Teaching/"
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
gbq_proj_id = 'umt-msba' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


### Pulling data from GBQ

Let's get the items and their categories from our Dram data.

In [3]:
query = """
    SELECT *
    FROM `umt-msba.dram_shop.brewery_data`
"""

brewery_data = client.query(query).to_dataframe()

In [4]:
query = """
    SELECT item,
           category, 
           price_point_name, 
           EXTRACT(YEAR FROM CAST(date as DATE)) AS year,
           SUM(gross_sales) AS gross_sales
    FROM `umt-msba.dram_shop.dram_items_2022*`
    GROUP BY item, category, price_point_name, year
"""

item_info = client.query(query).to_dataframe()


In [5]:
item_info.category.value_counts()

F-IPA Draught                  579
C-IPA Draught                  559
F-Lagers/Pils/Wheat Draught    240
C-Lagers/Pils/Wheat Draught    237
F-Seasonal                     234
                              ... 
Wine - Bottled                   1
Double K                         1
zCider - Draught                 1
Big Beer - Bottled               1
zIPA - Draught                   1
Name: category, Length: 81, dtype: int64

In [6]:
item_info.price_point_name.value_counts()

Half Glass                   690
64 oz                        450
Pint                         306
To-Go                        297
32 oz                        250
                            ... 
XXL                            1
Flight of 3                    1
Glacier brewing root beer      1
White Wine Glass 12oz          1
Gray                           1
Name: price_point_name, Length: 180, dtype: int64

### Investigating Categories

Let's take a look at the categories and do some work to clean them up (mostly across locations)

In [7]:
# Going to assume lowercase

cat_prefix = re.compile(r"[fc] ?- ?")
cat_suffix = re.compile(r" -? ?(draught|bottled|bottles|cans)")

In [8]:
def clean_cat(cat) : 
    
    """
        Given a raw category from Dram, return a clean category. Usually this involves 
        removing the prefix and Draught/bottled at the end.
    
    """
    clean_cat = cat.lower()

    clean_cat = cat_prefix.sub("",clean_cat)
    clean_cat = cat_suffix.sub("",clean_cat)
    
    return(clean_cat)


In [9]:
item_info['clean_category'] = item_info['category'].map(clean_cat)

In [10]:
item_info.head()

,item,category,price_point_name,year,gross_sales,clean_category
0,LA CROIX,Soda - Bottled,HERE,2022,118.0,soda
1,Potato CHIPS!,Snacks,Bag,2022,2620.0,snacks
2,Stretch Sticker,Misc.,Regular,2022,56.0,misc.
3,Roadhouse Wilson IPA,IPA - Bottled,to Go!,2022,50.0,ipa
4,Sticker Black Bottom,Misc.,Regular,2022,49.0,misc.


In [11]:
(item_info.groupby('clean_category').
    sum('gross_sales').reset_index().
    sort_values('gross_sales',ignore_index=True,ascending=False))

,clean_category,year,gross_sales
0,ipa,2553786,219034.75
1,lagers/pils/wheat,1209156,104918.00
2,wine,525720,59280.25
3,cider,798690,49871.00
4,sour,721854,41824.75
5,seasonal,837108,31086.00
6,porter/stout,531786,28900.00
7,ambers/pales,487302,24430.25
8,amber/pale,374070,20286.00
9,kombucha,12132,16938.50


This looks pretty good. Potentially some more work to do if we wanted to collapse some of the wine categories. 

### Clean up beers

Based on our earlier code, let's try to clean up the items that are in the beer categories. Maybe we can figure out what those are first.

In [12]:
all_cats = (item_info.groupby('clean_category').
    sum('gross_sales').reset_index().
    sort_values('gross_sales',ignore_index=True,ascending=False)['clean_category'].tolist())

In [12]:
# I used this to spot the beers and wines

start = 20
end = 50

dict(zip(range(start,end),all_cats[start:end]))

{20: 'sparkling wine',
 21: 'softgoods',
 22: 'hard seltzer',
 23: 'white wine',
 24: 'na',
 25: 'double k',
 26: 'seltzer',
 27: 'special orders',
 28: 'porters/stouts',
 29: 'rosé wine',
 30: 'glassware',
 31: 'skin contact wine',
 32: 'beer packages/tastings',
 33: 'rosé',
 34: 'wild',
 35: 'belgians',
 36: 'hard seltzers',
 37: 'gluten free/reduced',
 38: '$6 token!',
 39: 'gluten free',
 40: 'misc.',
 41: 'catering!',
 42: 'hard kombucha',
 43: 'hard kombucha-bottles',
 44: 'seasonal/event',
 45: 'big beer',
 46: 'zcider',
 47: 'zipa'}

In [13]:
beer_clean_cats = []

for idx in [0,1,4,5,6,7,8,10,28,34,35,37,39] :
    beer_clean_cats.append(all_cats[idx])


In [13]:
# The order might change so hard-coding these in. 

beer_clean_cats = ['ipa',
 'lagers/pils/wheat',
 'sour',
 'seasonal',
 'porter/stout',
 'ambers/pales',
 'amber/pale',
 'belgian',
 'porters/stouts',
 'wild',
 'belgians',
 'gluten free/reduced',
 'gluten free']

In [14]:
wine_clean_cats = ['wine',
 'red wine',
 'white wine',
 'sparkling',
 'rosé wine',
 'skin contact wine',
 'rosé']

### Beer Cleaning

Now we're in a position to go over the items where `clean_category` is in the clean beer categories and try to 
get down to a clean item level. 

We'll make use of the breweries we pulled from Open Brewery DB.

In [18]:
breweries.to_csv("brewery_data.txt",sep="\t")

In [19]:
pandas_gbq.to_gbq(breweries, "umt-msba.dram_shop.brewery_data", project_id=gbq_proj_id,if_exists="replace")

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2557.50it/s]


In [16]:
brewery_data.tail()

,id,name,brewery_type,street,address_2,address_3,city,state,county_province,postal_code,country,longitude,latitude,phone,website_url,updated_at,created_at
8101,sufferfest-beer-company-san-francisco,Sufferfest Beer Company,proprietor,1770 UNION STREET,None,None,San Francisco,California,None,94123-3404,United States,-122.4283947,37.79807486,4152386431,http://www.sufferfestbeer.com,2022-08-20T02:56:08.975Z,2022-08-20T02:56:08.975Z
8102,bruehol-brewing-llc-benicia,Bruehol Brewing LLC,proprietor,4828 E 2nd St,None,None,Benicia,California,None,94510-1023,United States,-122.1275637,38.08086313,7073276768,http://www.bruehol.com,2022-08-20T02:56:08.975Z,2022-08-20T02:56:08.975Z
8103,napa-palisades-beer-company-napa,Napa Palisades Beer Company,proprietor,1000 Main St,None,None,Napa,California,None,94559-2645,United States,-122.285449,38.299359,7072961552,http://www.napapalisadesbeer.com,2022-08-20T02:56:08.975Z,2022-08-20T02:56:08.975Z
8104,headlands-brewing-company-mill-valley,Headlands Brewing Company,proprietor,None,None,None,Mill Valley,California,None,94941-2003,United States,-122.5449763,37.9060368,4158604226,http://www.headlandsbrewing.com,2022-08-20T02:56:08.975Z,2022-08-20T02:56:08.975Z
8105,high-water-brewing-stockton,High Water Brewing,proprietor,None,None,None,Stockton,California,None,95204-2943,United States,-121.2907796,37.9577016,5304409098,http://www.highwaterbrewing.com,2022-08-20T02:56:08.975Z,2022-08-20T02:56:08.975Z


In [22]:
breweries = brewery_data['name'].unique()
breweries = set(breweries)

In [23]:
beer_items = item_info[item_info['clean_category'].isin(beer_clean_cats)] 

In [24]:
years = beer_items['year'].unique().tolist()

years

[2022]

In [ ]:
# TODO: iterate over the years looking for new patterns. 

In [60]:
prefix_pattern = re.compile(r"^\d\d?[a-z]? ") # going to use on lowercase
spaced_hyphen_pattern = re.compile(r" - ")

def clean_beers(item) :
    
    item = item.lower()
    
    clean_item = prefix_pattern.sub("",item)
    
    # did we have a prefix? 
    prefix = clean_item != item
        
    # A persistent misspelling
    clean_item = clean_item.replace("windmere","widmer")
    
    pieces = spaced_hyphen_pattern.split(clean_item)
    
    beer = ""
    brewery = ""
    other_info = ""

    # A few possibilities at this point. 
    # 1. item = "Roadhouse Wilson IPA": Here Roadhouse is the brewery, Wilson IPA is the beer. 
    #    I think we should look through the len(pieces) == 1 items and see if there's a brewery
    #    lurking. Similar: "Mac and Jack's African Amber 12oz can"
    # 2. item = "Fremont Brewing - Golden Pilsner 12oz can": This will successfully split, with the 
    #    brewery in spot 0 and the beer (plus form factor) in spot 1. 
    # 3. item = "05M Amber - Dos Equis": Like 2, but this one has a prefix and the beer is in spot 0.
    #    Also, "10P IPA - Big Sky Brewing"
    # 4. item = "04M Modelo Especial": Has the prefix, but won't split
    
    if not prefix and len(pieces) == 1 :
        print(item)
        print(clean_item)
        print(pieces)
        print("-"*30)

    
    return(clean_item)
    

In [61]:
items = beer_items['item'].tolist()

In [62]:
for idx, item in enumerate(items) : 
    
    clean_beers(item)
    
    if (idx) > 150 :
        break
    

roadhouse wilson ipa
roadhouse wilson ipa
['roadhouse wilson ipa']
------------------------------
mac and jack's african amber 12oz can
mac and jack's african amber 12oz can
["mac and jack's african amber 12oz can"]
------------------------------
blackfoot single malt ipa
blackfoot single malt ipa
['blackfoot single malt ipa']
------------------------------
okocim o.k lager
okocim o.k lager
['okocim o.k lager']
------------------------------
bitburger pilsner can
bitburger pilsner can
['bitburger pilsner can']
------------------------------
new hokkaido (mountains walking) yamabiko japanese lager 16oz can
new hokkaido (mountains walking) yamabiko japanese lager 16oz can
['new hokkaido (mountains walking) yamabiko japanese lager 16oz can']
------------------------------
new hokkaido (mountains walking) yamabiko japanese lager 16oz can
new hokkaido (mountains walking) yamabiko japanese lager 16oz can
['new hokkaido (mountains walking) yamabiko japanese lager 16oz can']
------------------

In [ ]:
item_translation = dict()
# key = original item
# value = [clean_item_name, brewery (if present), remainder (if present)]

clean_items = set()

for item in items :
    
    clean_item = prefix_pattern.sub("",item).lower()
    
    clean_item = clean_item.replace("windmere","widmer")
    
    clean_items.add(clean_item)
    pieces = spaced_hyphen_pattern.split(clean_item)
    
    beer = ""
    brewery = ""
    other_info = ""
    
    # This next section tries to get the right values in the right 
    # places for beer/brewery/other stuff
    if len(pieces) > 1 : 
        pieces = [p.strip() for p in pieces]
        
        if len(pieces) == 2 :        
            if pieces[0] in brewery_set : 
                brewery = pieces[0]
                beer = pieces[1]
            
            
            else :
                brewery = pieces[1]
                beer = pieces[0]

                
        elif len(pieces) == 3 :
            if pieces[0] in brewery_set : 
                brewery, beer, other_info = pieces
            else :
                beer, brewery, other_info = pieces
        else : 
            if pieces[0] in brewery_set : 
                brewery, beer = pieces[:2]
            else :
                beer = pieces[0]
                brewery = pieces[1]
                
            other_info = " - ".join(pieces[1:]).strip()
        
    else :
        
        clean_item_tokens = clean_item.split()
        
        
        
        for bry in brewery_set :
            if bry in clean_item : 
                brewery = bry
                beer = clean_item.replace(brewery,"")
                print(f"Brewery = {brewery}; Beer = {beer}")
            
    
#        for token in clean_item_tokens :
#            if token in brewery_set :
#                brewery = token
        
#        beer = " ".join([token for token in clean_item_tokens if token != brewery])
                
            
    item_translation[item] = [beer, brewery, other_info]

    if "Cioke" in item :
        print(item)
        print(item_translation[item])



In [59]:
for b in breweries : 
    if 'roadhouse' in b.lower() :
        print(b)

Roadhouse Pub and Eatery
Roadhouse Brewing Company
McMenamins Cornelius Pass Roadhouse


In [43]:
breweries

{'Forward Operating Base Brewing Company / FOB Brewing',
 'Point Labaddie Brewery',
 'Old Market Pub and Brewery',
 'Farmington Underground Canning and Kegging',
 'Dragon Hops Brewing',
 'Bandwagon Brewery',
 'Banded Brewing Company',
 'Park City Brewery',
 'Four Quarters Brewing Co',
 'Backswing Brewing Co.',
 'Steele and Hops Public House',
 'All Saints Brewing Co',
 'Orange County Brewers',
 'From The Earth Brewing Company',
 'Two Goats Brewing',
 'Durango Brewing Co',
 'Fyne Ales',
 'Chimera Brewing Company',
 'Right Around The Corner',
 'Sugarfoot Saloon',
 'In the Shadow Brewing',
 'At Large Brewing',
 'Diamond Mountain Brewery',
 'Here and Now Brewing Co',
 'Spangalang Brewery',
 'Crue Brew Brewery',
 'Farnam House Brewing Company',
 'Brew 32',
 'SanTan Brewing Co - Uptown Chandler',
 'Ludlam Island Brewery',
 'Belle Isle Restaurant and Brewing Co',
 'Dimensional Brewing Co.',
 'Black Sky Brewery',
 'Rockslide Brewing Co',
 'Gordon Biersch Brewery Restaurant - Annapolis',
 'Paon